In [ ]:
# !pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip install simpletransformers
# !pip install -q -U ipykernel



In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# !nvidia-smi 
!ps -ax | grep jupyter
# !sudo kill PID

     47 ?        Sl     0:01 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --LargeFileManager.delete_to_trash=False --MappingKernelManager.root_dir="/content"
    270 ?        Ssl    0:01 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-352e3919-4303-4dc0-a3bf-b85074e35325.json
    335 ?        S      0:00 /bin/bash -c ps -ax | grep jupyter
    337 ?        R      0:00 grep jupyter


In [ ]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel
!jupyter --version

     |████████████████████████████████| 788kB 20.2MB/s 
     |████████████████████████████████| 368kB 54.7MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.23.0 which is incompatible.
jupyter core     : 4.5.0
jupyter-notebook : 5.2.2
qtconsole        : 4.5.2
ipython          : 5.5.0
ipykernel        : 4.10.1
jupyter client   : 5.3.1
jupyter lab      : not installed
nbconvert        : 5.5.0
ipywidgets       : 7.5.0
nbformat         : 4.4.0
traitlets        : 4.3.2


In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()
!nvidia-smi

Sat May  1 01:30:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os, logging, csv
from datetime import datetime
import simpletransformers, warnings
import pandas as pd
from simpletransformers.config.model_args import Seq2SeqArgs
from simpletransformers.seq2seq import Seq2SeqModel
from simpletransformers.t5 import T5Model
from simpletransformers.t5 import T5Model, T5Args

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)


##############################################################

train_df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/translated_train.tsv", sep="\t", quoting=csv.QUOTE_NONE).astype(str)
eval_df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/test_2k.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

In [ ]:
train_df

,input_text,target_text,prefix
1,Die NBA-Saison 1975 - 76 war die 30. Saison de...,Die Saison 1975 - 76 der National Basketball A...,paraphrase
3,Wenn vergleichbare Flussraten aufrechterhalten...,"Die Ergebnisse sind hoch, wenn vergleichbare F...",paraphrase
4,Es ist der Sitz des Bezirks Zerendi in der Reg...,Es ist der Sitz des Bezirks Zerendi in der Reg...,paraphrase
5,William Henry Henry Harman wurde am 17. Februa...,William Henry Harman wurde am 17. Februar 1828...,paraphrase
7,Bei einer diskreten Anzahl von Wahrscheinlichk...,In Anbetracht einer diskreten Menge von Wahrsc...,paraphrase
...,...,...,...
49384,"Die romanische Sprache Galizisch (Galego), die...","Die romanische Sprache, die derzeit in Galizie...",paraphrase
49390,"Man beachte, dass k ein Vektor ist, der aus dr...","Es ist zu beachten, dass k ein Vektor ist, der...",paraphrase
49393,"Tim Henman gewann im Finale 6: 2, 7 - 6 gegen ...","Tim Tim Henman gewann im Finale 6 - 2, 7 - 6 g...",paraphrase
49395,Er galt als aktives Mitglied des Rates und wur...,Er galt als aktives Mitglied des Rates und wur...,paraphrase



**Training the actual model**

model parameter

In [ ]:
model_args = T5Args()
model_args.do_sample = True # Greedy search or samples
model_args.eval_batch_size = 4
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.preprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with T5 base"

training the model


In [ ]:
model = T5Model(
    "t5",
    "t5-base", 
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )


INFO:filelock:Lock 139690178796176 acquired on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

INFO:filelock:Lock 139690178796176 released on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock
INFO:filelock:Lock 139692830136208 acquired on /root/.cache/huggingface/transformers/ab4e948915b067f5cb6e5105f6f85044fd717b133f43240db67899a8fc7b29a2.26934c75adf19ceac3c268b721ba353356b7609c45f5627550326f275a2163b4.lock


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

INFO:filelock:Lock 139692830136208 released on /root/.cache/huggingface/transformers/ab4e948915b067f5cb6e5105f6f85044fd717b133f43240db67899a8fc7b29a2.26934c75adf19ceac3c268b721ba353356b7609c45f5627550326f275a2163b4.lock
INFO:filelock:Lock 139690183638672 acquired on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_128895
INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.8587442738935351}
INFO:simpletransformers.t5.t5_model: Training of t5-base model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/224 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:63: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f0c36ff3f80> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))


Decoding outputs:   0%|          | 0/2685 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:63: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f0c36fc0170> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:63: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f0c36fc0320> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:63: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f0c36fc04d0> on tensor will not be serialized.  If this is exp

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel
from simpletransformers.t5 import T5Model


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = T5Model(
    "t5", model_name="/content/drive/MyDrive/Colab Notebooks/T5_base/outputs"
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Enter text to paraphrase: Ich möchte mich mit großem Interesse um die Stelle als Business Analyst*in in XYZ bewerben. In den letzten fünf Jahren habe ich als Office und Business Manager die Berufserfahrung gesammelt, die für diese Stelle gesucht wird 


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/3 [00:00<?, ?it/s]

---------------------------------------------------------
Ich möchte mich mit großem Interesse um die Stelle als Business Analyst*in in XYZ bewerben. In den letzten fünf Jahren habe ich als Office und Business Manager die Berufserfahrung gesammelt, die für diese Stelle gesucht wird 

Predictions >>>
Ich möchte mich mit großem Interesse für die Position als Business Analyst*in in XYZ bewerben. In den vergangenen fünf Jahren habe ich als Office- und Geschäftsführer die Berufserfahrung gesammelt, die für diese Stelle gesucht wird.
In den letzten fünf Jahren habe ich die Berufserfahrung als Büro- und Business-Manager gesammelt, die für diesen Job gesucht wird.
Ich will mich auf den XYZ Business Analyst*-Fall begeben. Ich habe als Office- und Business Manager in den vergangenen fünf Jahren Berufserfahrung gesammelt, die für diesen Stellentyp gesucht wird.
---------------------------------------------------------

Enter text to paraphrase: Ich bin sehr an der Stelle als Project Manager*in zu

...

---------------------------------------------------------
Ich bin sehr an der Stelle als Project Manager*in zum nächstmöglichen Eintrittsdatum interessiert. Angesichts der Jobanforderungen, die Sie in der Stellenbeschreibung angegeben haben, glaube ich, dass ich der / die perfekte Kandidat*in für die Stelle bin. 

Predictions >>>
Ich bin sehr an der Stelle als Project Manager*in zum nächstmöglichen Eintrittsdatum interessiert. Angesichts der Stellenbeschreibung, die Sie in der Stellenbeschreibung angegeben haben, glaube ich, dass ich der / die perfekte Kandidat*in für die Stelle bin.
Ich bin sehr an der Stelle als Project Manager in der nächsten Bewerbung interessiert. Angesichts der Jobanforderungen, die Sie in der Stellenbeschreibung aufgeführt haben, glaube ich, dass ich der / die perfekte Kandidatin für die Stelle bin.
Ich bin sehr an der Position als Projektmanager*in für möglichen Eintrittsdatum interessiert. Angesichts der Jobanforderungen, die Sie in der Stellenbeschreibung ang

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/3 [00:00<?, ?it/s]

---------------------------------------------------------
Mit Begeisterung habe ich die offene Stellenanzeige in der Verwaltung von XYZ gelesen. Angesichts der Jobanforderungen, die Sie in der Stellenbeschreibung angegeben haben und der Erfahrung, die ich mitbringe, bin ich überzeugt, der / die perfekte Kandidat*in zu sein. 

Predictions >>>
Angesichts der Inanspruchnahme der Stellenbeschreibung und der Erfahrungen, die ich mitbringe, bin ich überzeugt, der / die perfekte Kandidatin* ist.
Angesichts der Stellenanforderungen, die sie in der Stellenbeschreibung geschrieben haben und der Erfahrung, die ich mitbringe, bin ich überzeugt, der / die perfekte Kandidatin*in zu sein.
Mit Begeisterung habe ich die offene Stellenanzeige in der Verwaltung von XYZ gelesen. Angesichts der Jobanforderungen, die Sie in der Stellenbeschreibung angegeben haben, und der Erfahrung, die ich mitbringe, bin ich überzeugt, dass der perfekte Kandidat*in zu sein ist.
---------------------------------------------

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
